In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
#%matplotlib inline
import seaborn as sns
import warnings
import datetime
import os
warnings.simplefilter("ignore")
from azure.storage.blob import BlockBlobService
import pyodbc
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import  linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from itertools import product
import statsmodels.api as sm
import psutil
import gc
from pyearth import Earth
from io import StringIO, BytesIO
verbose=True
savelocally=False
today = datetime.date.today()
month=today.month
year=today.year
metrics=[]

version=1.5
# V1.2 : Adapted script for new server classification method
# V1.2 : Changed parameter collection to be a list of lists rather than a dictionary
# V1.2 : Removed removeoutlier and serverclassification functions as they're no longer needed
# V1.5 : Adapted getinputfile_fromblob to read from blob but not save locally
# V1.5 : Changed folder path for push to blob functions
# V1.6 : By default all pushtoblob functions will not save locally unless savelocally is set to True

# V1.2 : changed file patterns for new file and removed parameter file pattern

def getblobfilename(tenant,role,asofdate):
    blobdatafilepattern = "{:#tenant#/efficiencyview/%Y/%m/Classifiedinput/#tenant#_Classified_Inputdata_for_rps_prediction_#role#_%Y%m%d.csv}"
    blobdatafile= blobdatafilepattern.format(asofdate).replace('#role#',role).replace('#tenant#',tenant)
    return blobdatafile

def makedirectory(tenant,role,asofdate):
    dirpath = "{:./efficiencyview/#tenant#/#role#/%Y/%m/}"
    path= dirpath.format(asofdate).replace('#role#',role).replace('#tenant#',tenant)
    if not os.path.exists(path):
        os.makedirs(path)
    return path

def getinputfile_fromblob(storageaccountname,accountkey,container,tenant,role,asofdate):
    byte_stream = BytesIO()
    blobservice = BlockBlobService(storageaccountname,accountkey)
    blobdatafile=getblobfilename(tenant,role,asofdate)
    blobservice.get_blob_to_stream(container_name=container, blob_name=blobdatafile, stream=byte_stream)
    byte_stream.seek(0)
    dfdata=pd.read_csv(byte_stream)
    byte_stream.close()
    return dfdata

def correlation(Data):
    datacorr=Data[['Requests_sec','PercentProcessorTime','RequestExecutionTime','Percent_Memory_Utilized']]
    corr = datacorr.corr()
    datacorr.corr()
    return datacorr.corr()

# V1.2 : The new data already has a date column - this function creates date variables that are used in data formatting
def dates(df):
    data_dates= pd.to_datetime(df['Interval'], format = "%Y-%m-%d")
    metrics.append(['number of unique dates',data_dates.dt.normalize().nunique()])
    mindate= data_dates.min().strftime('%Y-%#m-%d')
    maxdate= data_dates.max().strftime('%Y-%#m-%d')
    month =data_dates.min().strftime('%b')
    year = data_dates.min().strftime('%Y')
    daterange=month + " " + year +"("+ month +")"
    return mindate, maxdate, month, year, daterange

# V1.2 : created a function to collect raw data info to clean efficiency function
def rawdatametrics(df):
    metrics.append(['number of raw data rows', df.shape[0]])
    metrics.append(['number of unique data centers', df.DataCenter.nunique()])
    metrics.append(['number of unique active servers', df.ServerName.nunique()])
    metrics.append(['server classification method', "TotalTransactions"])

# V1.2 : created a function to create RAM column to clean efficiency view
def addmemoryutilized(df):
    maxmemory=df['AvailableMBytes'].max()
    df['Percent_Memory_Utilized']=((maxmemory-df['AvailableMBytes'])/maxmemory)*100
    return df

def filterbySTD(Data):    
    Data['RPS2']=np.where(Data['Requests_sec']==0,0.00001,Data['Requests_sec'])
    Data['PercentProcessorTime']=np.where(Data['PercentProcessorTime']==0,0.00001,Data['PercentProcessorTime'])
    Data['rt']=Data['RPS2']/Data['PercentProcessorTime']
    Data=Data[(Data['Requests_sec'] !=0)&(Data['RequestExecutionTime']!=0)]
    x, _ = stats.boxcox(Data['rt'])
    Data=Data.drop(['RPS2','rt'], axis=1)
    gc.collect()
    Data['tp']=x
    median=np.median(Data['tp'])
    sd = np.std(Data['tp'])
    Data['sd_rt']=(Data['tp']-median)/sd
    Data['sd_range2']=round(Data['sd_rt'],1)
    Data=Data.drop(['tp','sd_rt'], axis=1)
    gc.collect()
    if(verbose):
        print("% of Data between standard deviation range -1 & 1 :",len(Data[(Data['sd_range2']>-1)&(Data['sd_range2']<1)])/len(Data))
        print("% of Data between standard deviation range -2 & 2 :",len(Data[(Data['sd_range2']>-2)&(Data['sd_range2']<2)])/len(Data))
        print("% of Data between standard deviation range -3 & 3 :",len(Data[(Data['sd_range2']>-3)&(Data['sd_range2']<3)])/len(Data))
    Data['Outlier']=np.where((Data['sd_range2']<=-3)|(Data['sd_range2']>=3),1,0)
    cleanedData=Data[Data['Outlier']==0]
    if(verbose):
        print("Number of rows after cleaning data:",cleanedData.shape[0])
    cleanedData=cleanedData.drop(['Outlier','sd_range2'], axis=1)
    metrics.append(['number of rows after filtering by stdev', cleanedData.shape[0]])
    return cleanedData

# V1.1 : Changed X_train to df for memory optimization. This is reducing the memory required by not creating an extra data frame X_train
def filterbyInfluence(df, random_State, SamplePercent,path): 
    y=df[['Requests_sec']]
    y=pd.DataFrame(y)
    df=df[['PercentProcessorTime','Percent_Memory_Utilized','RequestExecutionTime','TotalTransactions']]
    test_size=(1-SamplePercent)
    df, X_test, y_train, y_test = train_test_split(df, y, test_size=test_size, random_state=random_State)
    clf= sm.OLS(y_train, sm.add_constant(df[['PercentProcessorTime','Percent_Memory_Utilized','RequestExecutionTime']])).fit()
    df['model_leverage'] = clf.get_influence().hat_matrix_diag
    df['model_cooks'] = clf.get_influence().cooks_distance[0]
    df=pd.concat([df, y_train], axis=1, join ='inner')
    sampletestdata=pd.concat([X_test, y_test], axis=1, join ='inner')
    
    #inf_data = df[(df.model_leverage >= (2/df.shape[0])) & (df.model_cooks >= 3*(df.model_cooks.mean())) > (df.model_cooks <= 1)]
    #inf_data.to_csv(os.path.join(path,'inf_data.csv'),index=False)
    df = df[(df.model_leverage < (2/df.shape[0])) | (df.model_cooks < 3*(df.model_cooks.mean())) | (df.model_cooks <= 1)]

    df=df.drop(['model_leverage','model_cooks'], axis=1)
    metrics.append(['random_state',random_State])
    metrics.append(['number of rows after filtering by influence', (df.shape[0] + sampletestdata.shape[0])])
    metrics.append(['number of sample rows', df.shape[0]])
    metrics.append(['sample dataset percentage', Sample_Percent])
    if(verbose):
        print("Number of rows after filtering by influence:",(df.shape[0] + sampletestdata.shape[0]))
    if (verbose):
        print('sample count',df.shape[0])
    return df,sampletestdata

# V1.1 : Eliminated the creation of new data frame X_train and X_test for memory.
# V1.1 : Removed the quantile stats as they are getting collected as a seperate function.
# V1.1 : Restructured the code for memory optimization.
# V1.1 : Rounded off the values of RET and Memory steps and end values for efficiency preditions.
# V1.2 : Still using linear regression for now - will change this function when we finalize which model to use
# V1.2 : Split the function into two - one to fit the model and another to create prediction data. sample_linear_regression now returned a fitted model
def linear_regression(df, sampletestdf, random_State, path,prediction,cv): 
    modeltype='LR'
    y_train = df[['Requests_sec']]
    sampletraindf = df[['PercentProcessorTime','Percent_Memory_Utilized','RequestExecutionTime']]
    gc.collect()
    y_true= sampletestdf['Requests_sec'].as_matrix()
    gc.collect()
    fitted = linear_model.LinearRegression(fit_intercept=True, normalize=True, copy_X=False, n_jobs=-1)
    fitted.fit(sampletraindf, y_train)
    sampletestdf['Requests_sec_LR'] = fitted.predict(sampletestdf[['PercentProcessorTime','Percent_Memory_Utilized','RequestExecutionTime']])
    prediction['RPS']=fitted.predict(prediction[['PercentProcessorTime','Percent_Memory_Utilized','RequestExecutionTime']])
    gc.collect()
    prediction['RPS']=np.where(prediction['RPS'] < 0,0,prediction['RPS'])
    fitted_cv = linear_model.LinearRegression(fit_intercept=True, normalize=True, copy_X=False, n_jobs=-1)
    cvMSE=-np.mean(cross_val_score(fitted_cv,sampletraindf[['PercentProcessorTime','Percent_Memory_Utilized','RequestExecutionTime']] , y_train , cv=cv, scoring='neg_mean_squared_error'))
    metrics.append(['sample cv MSE', cvMSE])
    r2=np.mean(cross_val_score(fitted_cv,sampletraindf[['PercentProcessorTime','Percent_Memory_Utilized','RequestExecutionTime']] , y_train , cv=cv, scoring='r2'))
    metrics.append(['sample cv R2',r2])
    fitted_cv = None
    gc.collect()
    MAE_LR=mean_absolute_error(y_true, sampletestdf['Requests_sec_LR'])
    metrics.append(['sample MAE LR',MAE_LR])
    MSE_LR=np.mean((sampletestdf['Requests_sec_LR'] - y_true) ** 2)
    metrics.append(['sample MSE LR',MSE_LR])
    RMSE_LR=np.sqrt(mean_squared_error(y_true, sampletestdf['Requests_sec_LR']))
    metrics.append(['sample RMSE LR',RMSE_LR])
    R2_LR=r2_score(y_true, sampletestdf['Requests_sec_LR'])
    metrics.append(['sample R2 LR',R2_LR])
    MAPE_LR=np.mean(np.abs((y_true - sampletestdf['Requests_sec_LR']) / y_true))
    metrics.append(['sample MAPE LR',MAPE_LR])
    metrics.append(['fitting model', modeltype])
    metrics.append(['number of sample train rows', sampletraindf.shape[0]])
    metrics.append(['number of sample test rows', sampletestdf.shape[0]])
    metrics.append(['sample test data size', round(sampletestdf.shape[0]/ (sampletraindf.shape[0] + sampletestdf.shape[0]), 2)])
    params=fitted.get_params()
    params_keys = list(params.keys())
    params_items = list(params.values())
    params_keys = ["model parameter "+ x for x in params_keys]
    params=list(zip(params_keys, params_items))
    [metrics.append(list(elem)) for elem in params]
    fitted = None

    if(verbose):
        print('cross val mse: %f' %cvMSE)
        print('R2: %.2f' %r2)
        print('mean absoluter error: %f' %mae)
        print('Mean squared error: %.2f' %mse)
    mpl.rcParams['agg.path.chunksize'] = 10000
    plt.scatter(sampletestdf['PercentProcessorTime'], y_test,  color='black')
    plt.plot(sampletestdf['PercentProcessorTime'], sampletestdf['Requests_sec_LR'], color='blue', linewidth=3)
    plt.xticks(())
    plt.yticks(())
    plotname="Plot_cleanData_vs_LR.png"
    plt.savefig(os.path.join(path,plotname))
    sampletestdf=None
    gc.collect()
    return prediction

def mars(df, sampletestdf, random_State, path, prediction): 
    modeltype='MARS'
    y_train = df[['Requests_sec']]
    sampletraindf = df[['PercentProcessorTime','Percent_Memory_Utilized','RequestExecutionTime']]
    gc.collect()
    y_true= sampletestdf['Requests_sec'].as_matrix()
    model_mars = Earth(max_terms=30)
    print("Fitting MARS model")
    model_mars.fit(sampletraindf,y_train)
    print(model_mars.summary())
    sampletestdf['Requests_sec_Mars'] = model_mars.predict(sampletestdf[['PercentProcessorTime','Percent_Memory_Utilized','RequestExecutionTime']])
    prediction['RPS']=model_mars.predict(prediction)
    model_mars= None
    gc.collect()
    prediction['RPS']=np.where(prediction['RPS'] < 0,0,prediction['RPS'])
    MAE_Mars=mean_absolute_error(y_true, sampletestdf['Requests_sec_Mars'])
    metrics.append(['sample MAE MARS',MAE_Mars])
    MSE_Mars=np.mean((sampletestdf['Requests_sec_Mars'] - y_true) ** 2)
    metrics.append(['sample MSE MARS',MSE_Mars])
    RMSE_Mars = np.sqrt(mean_squared_error(y_true, sampletestdf['Requests_sec_Mars']))
    metrics.append(['sample RMSE MARS',RMSE_Mars])
    R2_Mars = r2_score(y_true, sampletestdf['Requests_sec_Mars'])
    metrics.append(['sample R2 MARS',R2_Mars])
    MAPE_Mars=np.mean(np.abs((y_true - sampletestdf['Requests_sec_Mars']) / y_true))
    metrics.append(['sample MAPE MARS',MAPE_Mars])
    metrics.append(['sample random state', random_State])
    metrics.append(['fitting model',modeltype])
    metrics.append(['number of sample train rows', sampletraindf.shape[0]])
    metrics.append(['number of sample test rows', sampletestdf.shape[0]])
    metrics.append(['sample test data size', round(sampletestdf.shape[0]/ (sampletraindf.shape[0] + sampletestdf.shape[0]), 2)])
    gc.collect()
    if(verbose):
        print('R2: %.2f' %R2_Mars)
        print('mean absoluter error: %f' %MAE_Mars)
        print('Mean squared error: %.2f' %MSE_Mars)
    gc.collect()
    sampletestdf = sampletestdf[['PercentProcessorTime', 'Percent_Memory_Utilized','RequestExecutionTime', 'Requests_sec','Requests_sec_Mars']]
    sampletestdf.columns = ['CPU','Memory','RET','ActualRPS','RPSPredictMars']
    plt.scatter(sampletestdf['CPU'], sampletestdf['ActualRPS'],  color='black')
    plt.plot(sampletestdf['CPU'], sampletestdf['RPSPredictMars'], color='blue', linewidth=3)
    plt.xlabel("CPU")
    plt.ylabel("RPS")
    plotname="Plot_cleanData_vs_MARS.png"
    plt.savefig(os.path.join(path,plotname))
    plt.clf()
    sampletestdf=None
    gc.collect()
    return prediction

def comparemodels(sampletraindf, sampletestdf, prediction, SampleStartDate, SampleEndDate, tenant, role, random_State, path, cv):    
    y_train = sampletraindf[['Requests_sec']]
    sampletraindf = sampletraindf[['PercentProcessorTime','Percent_Memory_Utilized','RequestExecutionTime']]
    gc.collect()
    y_true= sampletestdf['Requests_sec'].as_matrix()
    metrics.append(['number of sample train rows', sampletraindf.shape[0]])
    metrics.append(['number of sample test rows',sampletestdf.shape[0]])
    metrics.append(['sample test data size', round(sampletestdf.shape[0]/ (sampletraindf.shape[0] + sampletestdf.shape[0]), 2)])
    metrics.append(['sample cv folds', cv])
    metrics.append(['sample random state', random_State])
    #MARSvsLR : Set verbose to >=1 to trace the model execution.
    #model_mars = Earth(max_terms=30,verbose=2)
    print("Fitting MARS model")
    model_mars = Earth(max_terms=30)
    model_mars.fit(sampletraindf,y_train)
    print(model_mars.summary())
    sampletestdf['Requests_sec_Mars'] = model_mars.predict(sampletestdf[['PercentProcessorTime','Percent_Memory_Utilized','RequestExecutionTime']])
    sampletestdf['Requests_sec_Mars'] = np.where(sampletestdf['Requests_sec_Mars'] < 0,0,sampletestdf['Requests_sec_Mars'])
    MAE_Mars=mean_absolute_error(y_true, sampletestdf['Requests_sec_Mars'])
    metrics.append(['sample MAE MARS',MAE_Mars])
    MSE_Mars=np.mean((sampletestdf['Requests_sec_Mars'] - y_true) ** 2)
    metrics.append(['sample MSE MARS',MSE_Mars])
    RMSE_Mars = np.sqrt(mean_squared_error(y_true, sampletestdf['Requests_sec_Mars']))
    metrics.append(['sample RMSE MARS',RMSE_Mars])
    R2_Mars = r2_score(y_true, sampletestdf['Requests_sec_Mars'])
    metrics.append(['sample R2 MARS',R2_Mars])
    MAPE_Mars=np.mean(np.abs((y_true - sampletestdf['Requests_sec_Mars']) / y_true))
    metrics.append(['sample MAPE MARS',MAPE_Mars])
    prediction['RPS_MARS']=model_mars.predict(prediction)
    prediction['RPS_MARS']=np.where(prediction['RPS_MARS'] < 0,0,prediction['RPS_MARS'])
    model_mars = None
    gc.collect()

    print("Fitting LR model")
    fitted = linear_model.LinearRegression(fit_intercept=True, normalize=True, copy_X=False, n_jobs=-1)
    fitted.fit(sampletraindf, y_train)
    params=fitted.get_params()
    params_keys = list(params.keys())
    params_items = list(params.values())
    params_keys = ["model parameter "+ x for x in params_keys]
    params=list(zip(params_keys, params_items))
    [metrics.append(list(elem)) for elem in params]
    sampletestdf['Requests_sec_LR'] = fitted.predict(sampletestdf[['PercentProcessorTime','Percent_Memory_Utilized','RequestExecutionTime']])
    sampletestdf['Requests_sec_LR'] = np.where(sampletestdf['Requests_sec_LR'] < 0,0,sampletestdf['Requests_sec_LR'])
    fitted_cv = linear_model.LinearRegression(fit_intercept=True, normalize=True, copy_X=False, n_jobs=-1)
    cvMSE=-np.mean(cross_val_score(fitted_cv,sampletraindf[['PercentProcessorTime','Percent_Memory_Utilized','RequestExecutionTime']] , y_train , cv=cv, scoring='neg_mean_squared_error'))
    metrics.append(['sample cv MSE LR',cvMSE])
    r2=np.mean(cross_val_score(fitted_cv,sampletraindf[['PercentProcessorTime','Percent_Memory_Utilized','RequestExecutionTime']] , y_train , cv=cv, scoring='r2'))
    metrics.append(['sample cv R2 LR',r2])
    MAE_LR=mean_absolute_error(y_true, sampletestdf['Requests_sec_LR'])
    metrics.append(['sample MAE LR',MAE_LR])
    MSE_LR=np.mean((sampletestdf['Requests_sec_LR'] - y_true) ** 2)
    metrics.append(['sample MSE LR',MSE_LR])
    RMSE_LR=np.sqrt(mean_squared_error(y_true, sampletestdf['Requests_sec_LR']))
    metrics.append(['sample RMSE LR',RMSE_LR])
    R2_LR=r2_score(y_true, sampletestdf['Requests_sec_LR'])
    metrics.append(['sample R2 LR',R2_LR])
    MAPE_LR=np.mean(np.abs((y_true - sampletestdf['Requests_sec_LR']) / y_true))
    metrics.append(['sample MAPE LR',MAPE_LR])
    prediction['RPS_LR']=fitted.predict(prediction[['PercentProcessorTime','Percent_Memory_Utilized','RequestExecutionTime']])
    prediction['RPS_LR']=np.where(prediction['RPS_LR'] < 0,0,prediction['RPS_LR'])
    fitted = None
    sampletraindf = None
    gc.collect()
 
    sampletestdf = sampletestdf[['PercentProcessorTime', 'Percent_Memory_Utilized','RequestExecutionTime', 'Requests_sec','Requests_sec_LR', 'Requests_sec_Mars']]
    sampletestdf.columns = ['CPU','Memory','RET','ActualRPS','RPSPredictLR','RPSPredictMars']
    #generate_test_data_summary(df=sampletestdf,SampleStartDate = SampleStartDate,SampleEndDate = SampleEndDate,cpubucket=2,memorybucket=2,retbucket=10,path=path,tenant=tenant,role=role)
    #enddate = SampleEndDate[:10].replace("-","")
    #sampletestdf['SampleStartDate'] = SampleStartDate
    #sampletestdf['SampleEndDate'] = SampleEndDate
    #sampletestdf['Tenant'] = tenant
    #sampletestdf['Role'] = role
    #sampletestfile = 'SampleTest'+role+enddate+'.csv'
    #sampletestdf.to_csv(os.path.join(path,sampletestfile),index=False)
    #error_metrics = pd.DataFrame([["Mars",MAE_Mars,MSE_Mars,RMSE_Mars,R2_Mars,MAPE_Mars],["LR",MAE_LR,MSE_LR,RMSE_LR,R2_LR,MAPE_LR]],columns=['Model','MAE','MSE','RMSE','R2','MAPE'])
    #errormetricsfile = 'ModelComparisionErrorMetrics'+role+enddate+'.csv'
    #error_metrics.to_csv(os.path.join(path,errormetricsfile),index=False)
    #sampletestdf = None
    gc.collect()
    if(verbose):
        print('cross val mse: %f' %cvMSE)
        print('R2: %.2f' %R2_LR)
        print('mean absoluter error: %f' %MAE_LR)
        print('Mean squared error: %.2f' %MSE_LR)
        
    #print("Plotting LR model")
    #mpl.rcParams['agg.path.chunksize'] = 10000
    #plt.scatter(sampletestdf['CPU'], sampletestdf['ActualRPS'],  color='black')
    #plt.plot(sampletestdf['CPU'], sampletestdf['RPSPredictLR'], color='blue', linewidth=3)
    #plt.xlabel("CPU")
    #plt.ylabel("RPS")
    #plotname="Plot_cleanData_vs_LR.png"
    #plt.savefig(os.path.join(path,plotname))
    #plt.clf()
    #plt.scatter(sampletestdf['CPU'], sampletestdf['ActualRPS'],  color='black')
    #plt.plot(sampletestdf['CPU'], sampletestdf['RPSPredictMars'], color='blue', linewidth=3)
    #plt.xlabel("CPU")
    #plt.ylabel("RPS")
    #plotname="Plot_cleanData_vs_MARS.png"
    #plt.savefig(os.path.join(path,plotname))
    #plt.clf()
    sampletestdf = None
    return prediction

# V1.2 : new function to create pseudo data
def generateprediction(df):
    RET_lower=round(np.percentile(df['RequestExecutionTime'],10),0)
    RET_upper=round(np.percentile(df['RequestExecutionTime'],90),0)
    RAM_lower=round(np.percentile(df['Percent_Memory_Utilized'],10),0)
    RAM_upper=round(np.percentile(df['Percent_Memory_Utilized'],90),0)
    RET_step = round((RET_upper-RET_lower)/10,0)
    RAM_step = round((RAM_upper-RAM_lower)/10,0)
    RET_step = np.where(RET_step == 0,1,RET_step)
    RAM_step = np.where(RAM_step == 0,1,RAM_step)
    #1.2 added step size to printed values
    if(verbose):
        print('RET_lower: %d' %RET_lower, 'RET_upper: %d' %RET_upper, 'RET_step: %d' %RET_step)
        print('RAM_lower: %d' %RAM_lower, 'RAM_upper: %d' %RAM_upper, 'RAM_step: %d' %RAM_step)
    PercentProcessorTime = np.arange(5, 85, 5)
    RequestExecutionTime = np.arange(RET_lower,(RET_upper+RET_step), RET_step)
    Percent_Memory_Utilized = np.arange(RAM_lower,(RAM_upper+RAM_step), RAM_step)
    PercentProcessorTime = PercentProcessorTime.astype('int')
    RequestExecutionTime = RequestExecutionTime.astype('int')
    Percent_Memory_Utilized = Percent_Memory_Utilized.astype('int')
    prediction={'PercentProcessorTime':PercentProcessorTime,'RequestExecutionTime':RequestExecutionTime,'Percent_Memory_Utilized':Percent_Memory_Utilized}
    prediction=pd.DataFrame([row for row in product(*prediction.values())],columns=prediction.keys())
    prediction=prediction[['PercentProcessorTime','Percent_Memory_Utilized','RequestExecutionTime']]
    return prediction 

# V1.2 : changed function name to be more descriptive and changed the date columns to just be assigned the min and max dates created earlier
def formatsampledata(df, SampleStartDate, SampleEndDate, DateRange, tenant, role):
    df['SampleStartDate']=SampleStartDate
    df['SampleEndDate']=SampleEndDate
    df['Tenant']=tenant.upper()
    df['Role']=role
    df['Region']='All'
    df['CPU']=df['PercentProcessorTime']
    df['MemoryAvailable']=df['AvailableMBytes']
    df['MemoryUtilized']=df['Percent_Memory_Utilized']
    df['RET']=df['RequestExecutionTime']
    df['RPS']=df['Requests_sec']
    df['DateRange']=DateRange
    df=df.drop(['PercentProcessorTime','AvailableMBytes','Percent_Memory_Utilized','RequestExecutionTime','Requests_sec'],axis=1)
    gc.collect()
    df=df[['SampleStartDate','SampleEndDate','Tenant','Role','Region','DataCenter','CPU','MemoryAvailable','MemoryUtilized','RET','RPS','DateRange']]
    gc.collect()
    return df

# V1.1 : instead of passing df(sample set), pass only the required date fields
# V1.2 : (same as above)changed function name to be more descriptive and changed the date columns to just be assigned the min and max dates created earlier
def formatpredictiondata(prediction,SampleStartDate,SampleEndDate,DateRange, tenant, role):
    prediction['SampleStartDate']=SampleStartDate
    prediction['SampleEndDate']=SampleEndDate
    prediction['Tenant']=tenant.upper()
    prediction['Role']=role
    prediction['Region']='All'
    prediction['FaultDomain']='All'
    prediction['CPU']=prediction['PercentProcessorTime']
    prediction['Memory']=prediction['Percent_Memory_Utilized']
    prediction['RET']=prediction['RequestExecutionTime']
    prediction['DateRange']=DateRange
    today=datetime.date.today().strftime('%Y-%#m-%d')
    prediction['PredictionDate']=today
    prediction=prediction[['SampleStartDate','SampleEndDate','Tenant','Role','Region','FaultDomain','CPU','Memory','RET','RPS_MARS', 'RPS_LR','DateRange','PredictionDate']]
    return prediction

# V1.2 : changed name and now takes min and max dates as input variables
def formatparameters (metrics, tenant, mindate, maxdate,role,path):
    analysis_info=pd.DataFrame(metrics, columns=['parameter','value'])
    analysis_info ['batch'] = tenant + "_" + role + "_" + maxdate
    analysis_info['tenant'] = tenant
    analysis_info['role'] = role
    analysis_info['batch_start_date'] = mindate
    analysis_info['batch_end_date'] = maxdate
    analysis_info = analysis_info[['batch','tenant','role','batch_start_date','batch_end_date', 'parameter', 'value']]
    return analysis_info

def PowerBISample(df,No_PBI_samples):
    df['CPU'] = round(df.PercentProcessorTime,1)
    df['RPS'] = round(df.Requests_sec,0)
    count = (df.groupby(['CPU','RPS']).count()).shape[0]
    count = round(No_PBI_samples/count)
    Power_BI_Sample = df.groupby(['CPU','RPS']).head(count)
    metrics.append(['number of PowerBI sample rows', Power_BI_Sample.shape[0]])
    metrics.append(['PowerBI dataset max', No_PBI_samples])
    if (verbose):
        print('power bi count',Power_BI_Sample.shape[0])
    return Power_BI_Sample
    
#### DISTRIBUTION FUNCTIONS


# V1.1 : Changed the distribution functions for memory optimization.
def cpu_memory_feature_distribution (Data,mindate,maxdate,bucket,tenant,role,SampleType,feature):
    if feature.lower() == 'cpu':
        bucket=bucket
        Data = Data[['PercentProcessorTime','TotalTransactions']]
        Data['range_1'] = (Data['PercentProcessorTime'] // bucket) * bucket
    if feature.lower() == 'memory':
        if 'Percent_Memory_Utilized' not in Data.columns:
            maxmemory = max(Data['AvailableMBytes'])
            Data['Percent_Memory_Utilized'] = ((maxmemory - Data['AvailableMBytes'])/maxmemory)*100
        bucket=bucket
        Data = Data[['Percent_Memory_Utilized','TotalTransactions']]
        Data['range_1'] = (Data['Percent_Memory_Utilized'] // bucket) * bucket
    #memory_availability()
    distribution = Data.groupby(['range_1'],as_index=False)['TotalTransactions'].agg({"Count": "count",
                                                       "TotalTransactions" : "sum"})
    #memory_availability()
    Data=None
    distribution = distribution.sort_values(by = 'range_1')
    distribution=distribution.reset_index(drop=True)
    distribution['range_2'] = np.where(distribution['range_1'] != 100,distribution['range_1'] + bucket ,distribution['range_1'])
    distribution['range_1'] = distribution[['range_1']].astype('int').astype('str')
    distribution['range_2'] = distribution[['range_2']].astype('int').astype('str')
    distribution['Bucket'] = distribution['range_1'] + '-' + distribution['range_2']
    nrecords = distribution["Count"].sum()
    totaltransactions = distribution["TotalTransactions"].sum()
    distribution['CountPercent'] = (distribution['Count']/nrecords)*100
    distribution['TransactionsPercent'] = (distribution['TotalTransactions']/totaltransactions)*100
    distribution['Tenant'] = tenant.upper()
    distribution['Role'] = role
    distribution['SampleStartDate'] = mindate
    distribution['SampleEndDate'] = maxdate
    distribution['SampleType'] = SampleType
    distribution=distribution.drop(['range_1','range_2'], axis=1)
    gc.collect()
    distribution = distribution[['SampleStartDate','SampleEndDate','SampleType','Tenant','Role','Bucket','Count','CountPercent','TotalTransactions','TransactionsPercent']]
    return distribution

# V1.1 : Changed the distribution functions for memory optimization.
def ret_rps_feature_distribution (Data,mindate,maxdate,tenant,role,SampleType,feature):
    #memory_availability()
    if feature.lower() == 'ret':
        Data = Data[['RequestExecutionTime','TotalTransactions']]
        Data = Data.sort_values(by=['RequestExecutionTime'],ascending=[True])
        Data = Data.reset_index(drop=True)
        Data['Percentile'] = pd.qcut(Data.index, 20, labels=False)
        distribution = Data.groupby(['Percentile'],as_index=False)['TotalTransactions'].agg({"Count": "count",
                                                           "TotalTransactions" : "sum"})
        distribution2 = Data.groupby(['Percentile'],as_index=False)['RequestExecutionTime'].agg({"range_1": "min",
                                                           "range_2" : "max"})
        distribution = pd.merge(distribution,distribution2,how="inner",on=['Percentile'])
    if feature.lower() == 'rps':
        Data = Data[['Requests_sec','TotalTransactions']]
        Data = Data.sort_values(by=['Requests_sec'],ascending=[True])
        Data = Data.reset_index(drop=True)
        Data['Percentile'] = pd.qcut(Data.index, 20, labels=False)
        distribution = Data.groupby(['Percentile'],as_index=False)['TotalTransactions'].agg({"Count": "count",
                                                           "TotalTransactions" : "sum"})
        distribution2 = Data.groupby(['Percentile'],as_index=False)['Requests_sec'].agg({"range_1": "min",
                                                           "range_2" : "max"})
        distribution = pd.merge(distribution,distribution2,how="inner",on=['Percentile'])
    #memory_availability()
    Data=None
    distribution = distribution.sort_values(by = 'range_1')
    distribution['range_1'] = distribution[['range_1']].astype('str')
    distribution['range_2'] = distribution[['range_2']].astype('str')
    distribution['Bucket'] = distribution['range_1'] + '-' + distribution['range_2']
    distribution['PercentileBucket'] = (distribution['Percentile']*5).astype('str') + '-' + (distribution['Percentile']*5+5).astype('str')
    distribution=distribution.reset_index(drop=True)
    nrecords = distribution["Count"].sum()
    totaltransactions = distribution["TotalTransactions"].sum()
    distribution['CountPercent'] = (distribution['Count']/nrecords)*100
    distribution['TransactionsPercent'] = (distribution['TotalTransactions']/totaltransactions)*100
    distribution['Tenant'] = tenant.upper()
    distribution['Role'] = role
    distribution['SampleStartDate'] = mindate
    distribution['SampleEndDate'] = maxdate
    distribution['SampleType'] = SampleType
    distribution=distribution.drop(['range_1','range_2'], axis=1)
    gc.collect()
    distribution = distribution[['SampleStartDate','SampleEndDate','SampleType','Tenant','Role','Bucket','Count','CountPercent','TotalTransactions','TransactionsPercent','Percentile','PercentileBucket']]
    return distribution    

# V1.1 : Changed the distribution functions for memory optimization.
# V1.1 : Added columns average RET and average Memory.
def cpuvsrps_Distribution(Data,mindate,maxdate,cpuvsrps_rps_limit,cpuvsrps_rpsbucket,tenant,role,SampleType):
    #print("cpurpsdist:")
    #memory_availability()
    Data['CPU'] = round(Data['PercentProcessorTime'],0)
    Data['RPS'] = (np.ceil(Data['Requests_sec']/cpuvsrps_rpsbucket)* cpuvsrps_rpsbucket) 
    Data=Data.drop(['PercentProcessorTime','Requests_sec'],axis=1)
    gc.collect()
    #memory_availability()
    if cpuvsrps_rps_limit == None:
        cpuvsrps_rps_limit = (np.ceil(max(Data['RPS'])/cpuvsrps_rpsbucket)* cpuvsrps_rpsbucket) - cpuvsrps_rpsbucket
    #memory_availability()
    Data['RPS'] = np.where((Data['RPS'] > cpuvsrps_rps_limit),cpuvsrps_rps_limit + cpuvsrps_rpsbucket ,Data['RPS'])
    CPU_RPS_distribution = Data.groupby(['CPU','RPS'],as_index=False)['TotalTransactions'].agg({"Count": "count","TotalTransactions" : "sum"})
    CPU_RPS_distribution2 = Data.groupby(['CPU','RPS'],as_index=False)['Percent_Memory_Utilized'].agg({"AverageMemory" : "mean"})
    CPU_RPS_distribution3 = Data.groupby(['CPU','RPS'],as_index=False)['RequestExecutionTime'].agg({"AverageRET" : "mean"})
    Data=None
    gc.collect()
    CPU_RPS_distribution = pd.merge(CPU_RPS_distribution,CPU_RPS_distribution2,how="inner",on=['CPU','RPS'])
    CPU_RPS_distribution2 = None
    gc.collect()
    CPU_RPS_distribution = pd.merge(CPU_RPS_distribution,CPU_RPS_distribution3,how="inner",on=['CPU','RPS'])
    CPU_RPS_distribution3 = None
    gc.collect()
    #memory_availability()
    CPU_RPS_distribution['CPU'] = CPU_RPS_distribution['CPU'].astype('int')
    CPU_RPS_distribution['RPS'] = CPU_RPS_distribution['RPS'].astype('int')
    CPU_RPS_distribution['CumulativeCount'] = CPU_RPS_distribution.groupby(['CPU'])['Count'].apply(lambda x: x.cumsum())
    CPU_RPS_distribution['TotalCount'] = CPU_RPS_distribution.groupby(['CPU'])['Count'].transform('sum')
    CPU_RPS_distribution['Percent'] = (CPU_RPS_distribution['CumulativeCount']/CPU_RPS_distribution['TotalCount'])*100
    CPU_RPS_distribution['RPSDescription'] = np.where((CPU_RPS_distribution['RPS'] > cpuvsrps_rps_limit),("> " + ((CPU_RPS_distribution['RPS']-cpuvsrps_rpsbucket).astype('str'))),("<= " + (CPU_RPS_distribution['RPS'].astype('str'))))
    CPU_RPS_distribution['RPSRange'] = np.where((CPU_RPS_distribution['RPS'] > cpuvsrps_rps_limit),("> " + ((CPU_RPS_distribution['RPS']-cpuvsrps_rpsbucket).astype('str'))),(np.maximum((CPU_RPS_distribution['RPS'] - cpuvsrps_rpsbucket).astype('int'),0).astype('str') + " - " + (CPU_RPS_distribution['RPS'].astype('str'))))
    CPU_RPS_distribution['Tenant'] = tenant.upper()
    CPU_RPS_distribution['Role'] = role
    CPU_RPS_distribution['SampleStartDate'] = mindate
    CPU_RPS_distribution['SampleEndDate'] = maxdate
    CPU_RPS_distribution['SampleType'] = SampleType
    gc.collect()
    CPU_RPS_distribution = CPU_RPS_distribution[['SampleStartDate','SampleEndDate','SampleType','Tenant','Role','CPU','RPS','Count','CumulativeCount','TotalCount','Percent','AverageMemory','AverageRET','TotalTransactions']]
    return(CPU_RPS_distribution)

# function to push distributions into blob
def push_distributions_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,df,SampleType,feature):
    blobpredictionfilepattern="{:#tenant#/efficiencyview/%Y/%m/distributions/#feature#/#tenant#_#role#_efficiencyview_#feature#_distribution_#SampleType#_%Y%m.csv}"
    blobservice = BlockBlobService(storageaccountname,accountkey)
    blobfile= blobpredictionfilepattern.format(asofdate).replace('#role#',role).replace('#tenant#',tenant).replace('#feature#', feature.lower()).replace('#SampleType#', SampleType.lower())
    data= StringIO()
    df.to_csv(data, index=False)
    data=bytes(data.getvalue(), 'utf-8')
    data=BytesIO(data)
    blobservice.create_blob_from_stream(container,blobfile,data)
    data.close()
    if (savelocally):
        localpredictionfilepattern="{:#tenant#_#role#_efficiencyview_#feature#_distribution_#SampleType#_%Y%m.csv}"
        localfile= localpredictionfilepattern.format(asofdate).replace('#role#',role).replace('#tenant#',tenant).replace('#feature#', feature.lower()).replace('#SampleType#', SampleType.lower())
        df.to_csv(os.path.join(path,localfile), index=False)

# V1.2 : put the dist_savefiles function within this distributions function so that only one function needs to be called in the efficiency view
def distributions(Data,mindate,maxdate,tenant,role,cpu_bucket,memory_bucket,cpuvsrps_rpsbucket,cpuvsrps_rps_limit,path,asofdate,SampleType):
    if(cpu_bucket == None):
        cpu_bucket=5
    if(memory_bucket == None):
        memory_bucket=5
    if(cpuvsrps_rpsbucket == None):
        cpuvsrps_rpsbucket=5
    gc.collect()
    ret_dist = ret_rps_feature_distribution(Data,mindate,maxdate,tenant,role,SampleType,feature='ret')
    rps_dist = ret_rps_feature_distribution(Data,mindate,maxdate,tenant,role,SampleType,feature='rps')
    cpu_dist = cpu_memory_feature_distribution(Data,mindate,maxdate,cpu_bucket,tenant,role,SampleType,feature='cpu')
    memory_dist = cpu_memory_feature_distribution(Data,mindate,maxdate,cpu_bucket,tenant,role,SampleType,feature='memory')
    cpurps_dist = cpuvsrps_Distribution(Data,mindate,maxdate,cpuvsrps_rps_limit,cpuvsrps_rpsbucket,tenant,role,SampleType)
    push_distributions_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,ret_dist,SampleType,feature='ret')
    push_distributions_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,rps_dist,SampleType,feature='rps')
    push_distributions_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,cpu_dist,SampleType,feature='cpu')
    push_distributions_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,memory_dist,SampleType,feature='memory')
    push_distributions_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,cpurps_dist,SampleType,feature='cpurps')
    gc.collect()

# function to push quantiles to the blob
def push_quantiles_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,df,SampleType):
    blobsamplefilepattern="{:#tenant#/efficiencyview/%Y/%m/quantiles/quantiles/#tenant#_#role#_efficiencyview_quantiles_#SampleType#_%Y%m.csv}"
    blobservice = BlockBlobService(storageaccountname,accountkey)
    blobfile= blobsamplefilepattern.format(asofdate).replace('#role#',role).replace('#tenant#',tenant).replace('#SampleType#', SampleType.lower())
    data= StringIO()
    df.to_csv(data, index=False)
    data=bytes(data.getvalue(), 'utf-8')
    data=BytesIO(data)
    blobservice.create_blob_from_stream(container,blobfile,data)
    data.close()
    if (savelocally):
        localsamplefilepattern="{:#tenant#_#role#_efficiencyview_quantiles_#SampleType#_%Y%m.csv}"
        localfile= localsamplefilepattern.format(asofdate).replace('#role#',role).replace('#tenant#',tenant).replace('#SampleType#', SampleType.lower())
        df.to_csv(os.path.join(path,localfile), index=False)

# function to push summarystats dataframe into blob
def push_summarystats_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,df,SampleType):
    blobsamplefilepattern="{:#tenant#/efficiencyview/%Y/%m/quantiles/summarystats/#tenant#_#role#_efficiencyview_summarystats_#SampleType#_%Y%m.csv}"
    blobservice = BlockBlobService(storageaccountname,accountkey)
    blobfile= blobsamplefilepattern.format(asofdate).replace('#role#',role).replace('#tenant#',tenant).replace('#SampleType#', SampleType.lower())
    data= StringIO()
    df.to_csv(data, index=False)
    data=bytes(data.getvalue(), 'utf-8')
    data=BytesIO(data)
    blobservice.create_blob_from_stream(container,blobfile,data)
    data.close()
    if (savelocally):
        localsamplefilepattern="{:#tenant#_#role#_efficiencyview_summarystats_#SampleType#_%Y%m.csv}"
        localfile= localsamplefilepattern.format(asofdate).replace('#role#',role).replace('#tenant#',tenant).replace('#SampleType#', SampleType.lower())
        df.to_csv(os.path.join(path,localfile), index=False)        

# V1.2 : put the quantiles_savefiles function within this quantilesandsummary function so that only one function needs to be called in the efficiency view
def quantilesandsummary (Data,mindate,maxdate,tenant,role,path,asofdate,quantile_i,SampleType):
    quantile_i = quantile_i/100
    quants = np.arange(0,1.0001,quantile_i)
    Quantiles = pd.DataFrame({'quantile' : quants ,'RequestExecutionTime': [Data['RequestExecutionTime'].quantile(x) for x in quants],
             'PercentProcessorTime': [Data['PercentProcessorTime'].quantile(x) for x in quants],
             'Requests_sec': [Data['Requests_sec'].quantile(x) for x in quants],
             'Percent_Memory_Utilized': [Data['Percent_Memory_Utilized'].quantile(x) for x in quants]})
    
    Quantiles['Tenant'] = tenant.upper()
    Quantiles['Role'] = role
    Quantiles['SampleStartDate'] = mindate
    Quantiles['SampleEndDate'] = maxdate
    Quantiles['SampleType'] = SampleType
    Metrics = ['Standard deviation','mean','median','min','max']
    summarystats = pd.DataFrame({'Metrics' : Metrics ,'RequestExecutionTime': [Data.RequestExecutionTime.std(),Data.RequestExecutionTime.mean(),Data.RequestExecutionTime.median(),Data.RequestExecutionTime.min(),Data.RequestExecutionTime.max()],
             'PercentProcessorTime': [Data.PercentProcessorTime.std(),Data.PercentProcessorTime.mean(),Data.PercentProcessorTime.median(),Data.PercentProcessorTime.min(),Data.PercentProcessorTime.max()],
             'Requests_sec': [Data.Requests_sec.std(),Data.Requests_sec.mean(),Data.Requests_sec.median(),Data.Requests_sec.min(),Data.Requests_sec.max()],
             'Percent_Memory_Utilized': [Data.Percent_Memory_Utilized.std(),Data.Percent_Memory_Utilized.mean(),Data.Percent_Memory_Utilized.median(),Data.Percent_Memory_Utilized.min(),Data.Percent_Memory_Utilized.max()]})
    
    summarystats['Tenant'] = tenant.upper()
    summarystats['Role'] = role
    summarystats['SampleStartDate'] = mindate
    summarystats['SampleEndDate'] = maxdate
    summarystats['SampleType'] = SampleType
    push_quantiles_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,Quantiles,SampleType)
    push_summarystats_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,summarystats,SampleType)
    gc.collect()

# function to push prediction set dataframe into blob
def push_pbiprediction_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,prediction):
    blobpredictionfilepattern="{:#tenant#/efficiencyview/%Y/%m/predictions/#tenant#_#role#_efficiencyview_predictionset_%Y%m.csv}"
    blobservice = BlockBlobService(storageaccountname,accountkey)
    blobfile= blobpredictionfilepattern.format(asofdate).replace('#role#',role).replace('#tenant#',tenant)
    if role.lower() in ['estsfe', 'restdirectoryservice', 'adminwebservice']:
        prediction = prediction.drop('RPS_LR', axis=1)
        prediction.rename(columns={'RPS_MARS':'RPS'}, inplace=True)
    else:
        prediction = prediction.drop('RPS_MARS', axis=1)
        prediction.rename(columns={'RPS_LR':'RPS'}, inplace=True)
    data= StringIO()
    prediction.to_csv(data, index=False)
    data=bytes(data.getvalue(), 'utf-8')
    data=BytesIO(data)
    blobservice.create_blob_from_stream(container,blobfile,data)
    data.close()
    if(savelocally):
        localpredictionfilepattern="{:#tenant#_#role#_efficiencyview_predictionset_%Y%m.csv}"
        localfile= localpredictionfilepattern.format(asofdate).replace('#role#',role).replace('#tenant#',tenant)
        prediction.to_csv(os.path.join(path,localfile), index=False)

# function to push sample set dataframe into blob
def push_pbisample_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,sample):
    blobsamplefilepattern="{:#tenant#/efficiencyview/%Y/%m/sampleset/#tenant#_#role#_efficiencyview_sampleset_%Y%m.csv}"
    blobservice = BlockBlobService(storageaccountname,accountkey)
    blobfile= blobsamplefilepattern.format(asofdate).replace('#role#',role).replace('#tenant#',tenant)
    data= StringIO()
    sample.to_csv(data, index=False)
    data=bytes(data.getvalue(), 'utf-8')
    data=BytesIO(data)
    blobservice.create_blob_from_stream(container,blobfile,data)
    data.close()
    if (savelocally):
        localsamplefilepattern="{:#tenant#_#role#_efficiencyview_sampleset_%Y%m.csv}"
        localfile= localsamplefilepattern.format(asofdate).replace('#role#',role).replace('#tenant#',tenant)
        sample.to_csv(os.path.join(path,localfile), index=False)
        
# function to push training data dataframe into blob
def push_trainingdata_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,sample):
    blobsamplefilepattern="{:#tenant#/efficiencyview/%Y/%m/trainingdata/#tenant#_#role#_efficiencyview_trainingdata_%Y%m.csv}"
    blobservice = BlockBlobService(storageaccountname,accountkey)
    blobfile= blobsamplefilepattern.format(asofdate).replace('#role#',role).replace('#tenant#',tenant)
    data= StringIO()
    sample.to_csv(data, index=False)
    data=bytes(data.getvalue(), 'utf-8')
    data=BytesIO(data)
    blobservice.create_blob_from_stream(container,blobfile,data)
    data.close()
    if (savelocally):
        localsamplefilepattern="{:#tenant#_#role#_efficiencyview_trainingdata_%Y%m.csv}"
        localfile= localsamplefilepattern.format(asofdate).replace('#role#',role).replace('#tenant#',tenant)
        sample.to_csv(os.path.join(path,localfile), index=False)

# function to push metrics dataframe into blob
def push_metrics_toblob(storageacountname,accountkey,tenant,role,container,asofdate,metrics):
    blobsamplefilepattern="{:#tenant#/efficiencyview/%Y/%m/metrics/#tenant#_#role#_efficiencyview_metrics_%Y%m.csv}"
    blobservice = BlockBlobService(storageacountname,accountkey)
    blobfile= blobsamplefilepattern.format(asofdate).replace('#role#',role).replace('#tenant#',tenant)
    data= StringIO()
    metrics.to_csv(data, index=False)
    data=bytes(data.getvalue(), 'utf-8')
    data=BytesIO(data)
    blobservice.create_blob_from_stream(container,blobfile,data)
    data.close()
    if (savelocally):
        localmetricsfilepattern="{:#tenant#_#role#_efficiencyview_metrics_%Y%m.csv}"
        localfile= localmetricsfilepattern.format(asofdate).replace('#role#',role).replace('#tenant#',tenant)
        metrics.to_csv(os.path.join(path,localfile), index=False)
    
# function to push both prediction set dataframes into blob
def push_bothpredictions_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,prediction):
    blobservice = BlockBlobService(storageaccountname,accountkey)
    #mars prediction
    prediction_mars=prediction
    prediction_mars = prediction_mars.drop('RPS_LR', axis=1)
    prediction_mars.rename(columns={'RPS_MARS':'RPS'}, inplace=True)
    prediction_mars['ModelType']='MARS'
    blobpredictionfilepattern_mars="{:#tenant#/efficiencyview/%Y/%m/mars_lr_predictions/#tenant#_#role#_efficiencyview_predictionset_mars_%Y%m.csv}"
    blobfile_mars= blobpredictionfilepattern_mars.format(asofdate).replace('#role#',role).replace('#tenant#',tenant)
    data_mars= StringIO()
    prediction_mars.to_csv(data_mars, index=False)
    data_mars=bytes(data_mars.getvalue(), 'utf-8')
    data_mars=BytesIO(data_mars)
    blobservice.create_blob_from_stream(container,blobfile_mars,data_mars)
    data_mars.close()
    #lr prediction
    prediction_lr=prediction
    prediction_lr = prediction_lr.drop('RPS_MARS', axis=1)
    prediction_lr.rename(columns={'RPS_LR':'RPS'}, inplace=True)
    prediction_lr['ModelType']='LR'
    blobpredictionfilepattern_lr="{:#tenant#/efficiencyview/%Y/%m/mars_lr_predictions/#tenant#_#role#_efficiencyview_predictionset_lr_%Y%m.csv}"
    blobfile_lr= blobpredictionfilepattern_lr.format(asofdate).replace('#role#',role).replace('#tenant#',tenant)
    data_lr= StringIO()
    prediction_lr.to_csv(data_lr, index=False)
    data_lr=bytes(data_lr.getvalue(), 'utf-8')
    data_lr=BytesIO(data_lr)
    blobservice.create_blob_from_stream(container,blobfile_lr,data_lr)
    data_lr.close()
    if (savelocally):
        localpredictionfilepattern_mars="{:#tenant#_#role#_efficiencyview_predictionset_mars_%Y%m.csv}"
        localfile_mars= localpredictionfilepattern_mars.format(asofdate).replace('#role#',role).replace('#tenant#',tenant)
        prediction_mars.to_csv(os.path.join(path,localfile_mars), index=False)
        localpredictionfilepattern_lr="{:#tenant#_#role#_efficiencyview_predictionset_lr_%Y%m.csv}"
        localfile_lr= localpredictionfilepattern_lr.format(asofdate).replace('#role#',role).replace('#tenant#',tenant)
        prediction_lr.to_csv(os.path.join(path,localfile_lr), index=False)
    
def memory_availability():
    values = psutil.virtual_memory()
    total = values.total /(1024.0 ** 3)
    used = values.used /(1024.0 ** 3)
    available = values.available /(1024.0 ** 3)
    print("Total memory = {}; used memory = {}; available memory = {}".format(total, used,available))

# V1.1 : Added new function parameters.
# V1.1 : Calling the getinputfilefrom blob in this method : saves memory
# V1.2 : Cleaned efficiency function so that most processes occur within functions
def efficiency(tenant,role,asofdate,storageaccountname,accountkey,container,No_PBI_samples,Sample_Percent,random_State,cv,cpu_bucket=None,memory_bucket=None,cpuvsrps_rpsbucket=None,cpuvsrps_rps_limit=None):
# Read data from blob
    print("Opening memory : ")
    memory_availability()
    path = makedirectory(tenant,role,asofdate)
    #for dev use below
    #container2='data'
    #storageaccountname2= 'identitytrafficstorage'
    #accountkey2= 'SAvw58YxX+8C0lO3uS26Ux3wIWcRE/zzjq6kqHJBhLXAcYa8vlhsCt7+p5NULjlcjEynYY1ZmkE3ecNrM5fu6Q=='
    #df = getinputfile_fromblob(storageaccountname2,accountkey2,container2,tenant,role,asofdate)
    #for prod use below
    df = getinputfile_fromblob(storageaccountname,accountkey,container,tenant,role,asofdate)
    #for gateway-auth
    #df = df[~df['Day'].isin(['3/1/2018','3/2/2018','3/3/2018','3/4/2018','3/5/2018'])]
    rawdatametrics(df)
    gc.collect()
    print("Memory after loading data : ")
    memory_availability()
# Generate date variables
    df=df.drop(['Tenant','Role'], axis=1)
    mindate, maxdate, month, year, daterange=dates(df)
    gc.collect()
    print("Memory after generating dates : ")
    memory_availability()
# Generate new variables
    df=addmemoryutilized(df)
    gc.collect()
# Split data
    df2 = df[['DataCenter','AvailableMBytes']]
    df = df[['PercentProcessorTime', 'Requests_sec','RequestExecutionTime','Percent_Memory_Utilized','TotalTransactions']]
# Generate distribution and quantile files for active server data
    #distributions(df,mindate,maxdate,tenant,role,cpu_bucket,memory_bucket,cpuvsrps_rpsbucket,cpuvsrps_rps_limit,path,asofdate,SampleType='ActiveServerData')
    #gc.collect()
    #quantilesandsummary(df,mindate,maxdate,tenant,role,path,asofdate,quantile_i=5,SampleType='ActiveServerData')
    #gc.collect()
    #print("Memory after generating server distribution data : ")
    #memory_availability()
# Remove outliers and filter by influence
    df = filterbySTD(df)
    gc.collect()
    print("Memory after filtering by standard deviation : ")
    memory_availability()
    df,sampletestdf = filterbyInfluence(df,random_State,Sample_Percent,path)
    gc.collect()
    print("Memory after filtering by influence and sampling : ")
    memory_availability()
# Generate distribution data for sample train data
    #distributions(df,mindate,maxdate,tenant,role,cpu_bucket,memory_bucket,cpuvsrps_rpsbucket,cpuvsrps_rps_limit,path,asofdate,SampleType='SampleTrainData')
    #gc.collect()
    #quantilesandsummary(df,mindate,maxdate,tenant,role,path,asofdate,quantile_i=5,SampleType='SampleTrainData')
    #gc.collect()
    #print("Memory after generating sample distribution data : ")
    #memory_availability()
# Generate prediction data
    prediction = generateprediction(df)
    gc.collect()
    print("Memory after building prediction data : ")
    memory_availability()
# Build the model
#    if role.lower() in ['estsfe', 'restdirectoryservice', 'adminwebservice']:
#        prediction = mars(df,sampletestdf, random_State,path, prediction)
#        gc.collect()
#        print("Memory after building the model : ")
#        memory_availability()
#    else:
#        prediction = linear_regression(df,sampletestdf, random_State,path,prediction,cv=5)
#        gc.collect()
#        print("Memory after building the model : ")
#        memory_availability()
    prediction = comparemodels(df, sampletestdf, prediction, mindate, maxdate, tenant, role, random_State, path, cv)
    gc.collect()
    print("Memory after building the model : ")
    memory_availability()
# Get parameters
    parameters = formatparameters(metrics, tenant, mindate, maxdate,role,path)
    gc.collect()
    print("Memory after saving parameters : ")
    memory_availability()
# Collect sample train data
    sampletestdf = None
    gc.collect()
    df = pd.merge(df,df2,left_index=True,right_index=True,how='inner')
    gc.collect()
    print("Memory after fetching train data : ")
    memory_availability()
# Generate power BI sample
    Powerbi_sample = PowerBISample(df, No_PBI_samples)
    gc.collect()
    print("Memory after generating power bi sample : ")
    memory_availability()
# Format the files for power BI
    Powerbi_sample = formatsampledata(Powerbi_sample, mindate,maxdate,daterange, tenant, role)
    df = formatsampledata(df,mindate,maxdate,daterange, tenant, role)
    gc.collect()
    prediction= formatpredictiondata(prediction,mindate,maxdate,daterange, tenant, role)
    gc.collect()
    print("closing memory : ")
    memory_availability()
    return df,Powerbi_sample,prediction,parameters,path

In [3]:
#config
import datetime
print(str(datetime.datetime.today()))
asofdate=datetime.datetime(2018, 3, 31,0,0,0,0)
cv=5
No_PBI_samples=700000
Sample_Percent=0.7
random_State=10
cpu_bucket=5
memory_bucket=5
cpuvsrps_rpsbucket=5
cpuvsrps_rps_limit=100

#prod details
container='data'
storageaccountname= 'identitytrafficstorage'
accountkey= 'SAvw58YxX+8C0lO3uS26Ux3wIWcRE/zzjq6kqHJBhLXAcYa8vlhsCt7+p5NULjlcjEynYY1ZmkE3ecNrM5fu6Q=='

#dev details
#container='data'
#storageaccountname= 'devidtrafficstorage'
#accountkey= '7LjK/HqpmpSffbsi03TXSw8Jv4F9OWMQ47/I3I5Trd8rvUd3N1lyHOapBG/WrL3mtj5Gmvpeo/0wUoT49HFOvQ=='

2018-04-19 22:23:11.479866


# MDODSV2

In [ ]:
print(str(datetime.datetime.today()))
tenant='msodsv2'
msodsv2roles = ['directoryproxy']#['becwebservice','adminwebservice','companymanager','msods-syncservice','restdirectoryservice','directoryproxy']
for i in msodsv2roles:
    print(str(datetime.datetime.today()))
    print("Producing Efficiency View for : " , i)
    role = i
    metrics=[]
    sample,Powerbi_sample,prediction,parameters,path = efficiency(tenant,role,asofdate,storageaccountname,accountkey,container,No_PBI_samples,Sample_Percent,random_State,cv,cpu_bucket,memory_bucket,cpuvsrps_rpsbucket,cpuvsrps_rps_limit)
    #push_bothpredictions_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,prediction)
    #push_pbiprediction_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,prediction)
    #push_pbisample_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,Powerbi_sample)
    #push_trainingdata_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,sample)
    push_metrics_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,parameters)
    gc.collect()
    print(str(datetime.datetime.today()))

# MSGraph

In [ ]:
print(str(datetime.datetime.today()))
tenant='msgraph'
msgraphroles = ['AGSFE']
for i in msgraphroles:
    print("Producing Efficiency View for : " , i)
    role = i
    metrics=[]
    sample,Powerbi_sample,prediction,parameters,path = efficiency(tenant,role,asofdate,storageaccountname,accountkey,container,No_PBI_samples,Sample_Percent,random_State,cv,cpu_bucket,memory_bucket,cpuvsrps_rpsbucket,cpuvsrps_rps_limit)
    push_bothpredictions_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,prediction)
    push_pbiprediction_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,prediction)
    push_pbisample_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,Powerbi_sample)
    push_trainingdata_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,sample)
    push_metrics_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,parameters)
    gc.collect()

# ESTS

In [ ]:
print(str(datetime.datetime.today()))
tenant='ests'
estsroles = ['ESTSFE']
for i in estsroles:
    print("Producing Efficiency View for : " , i)
    role = i
    metrics=[]
    sample,Powerbi_sample,prediction,parameters,path = efficiency(tenant,role,asofdate,storageaccountname,accountkey,container,No_PBI_samples,Sample_Percent,random_State,cv,cpu_bucket,memory_bucket,cpuvsrps_rpsbucket,cpuvsrps_rps_limit)
    push_bothpredictions_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,prediction)
    #push_pbiprediction_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,prediction)
    push_pbisample_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,Powerbi_sample)
    push_trainingdata_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,sample)
    push_metrics_toblob(storageacountname,accountkey,tenant,role,container,asofdate,parameters)
    gc.collect()

# Gateway

In [3]:
print(str(datetime.datetime.today()))
tenant='gateway'
gatewayroles = ['gateway-auth']#, 'gateway-nonauth']
for i in gatewayroles:
    print("Producing Efficiency View for : " , i)
    role = i
    metrics=[]
    sample,Powerbi_sample,prediction,parameters,path = efficiency(tenant,role,asofdate,storageaccountname,accountkey,container,No_PBI_samples,Sample_Percent,random_State,cv,cpu_bucket,memory_bucket,cpuvsrps_rpsbucket,cpuvsrps_rps_limit)
    #push_bothpredictions_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,prediction)
    #push_pbiprediction_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,prediction)
    #push_pbisample_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,Powerbi_sample)
    #push_trainingdata_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,sample)
    push_metrics_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,parameters)
    gc.collect()

2018-04-18 16:29:49.282453
Producing Efficiency View for :  gateway-auth
Opening memory : 
Total memory = 127.99956130981445; used memory = 18.252113342285156; available memory = 109.7474479675293
Memory after loading data : 
Total memory = 127.99956130981445; used memory = 33.80404281616211; available memory = 94.19551849365234
Memory after generating dates : 
Total memory = 127.99956130981445; used memory = 31.777774810791016; available memory = 96.22178649902344
% of Data between standard deviation range -1 & 1 : 0.6614726406238697
% of Data between standard deviation range -2 & 2 : 0.9482523704774488
% of Data between standard deviation range -3 & 3 : 0.997002122625989
Number of rows after cleaning data: 134339060
Memory after filtering by standard deviation : 
Total memory = 127.99956130981445; used memory = 27.608821868896484; available memory = 100.39073944091797
Number of rows after filtering by influence: 134339048
sample count 94037329
Memory after filtering by influence and 

In [ ]:
memory_availability()

In [10]:
print(str(datetime.datetime.today()))
tenant='aduxp'
aduxproles = ['ADDataService']
for i in aduxproles:
    print("Producing Efficiency View for : " , i)
    role = i
    metrics=[]
    sample,Powerbi_sample,prediction,parameters,path = efficiency(tenant,role,asofdate,storageaccountname,accountkey,container,No_PBI_samples,Sample_Percent,random_State,cv,cpu_bucket,memory_bucket,cpuvsrps_rpsbucket,cpuvsrps_rps_limit)
    push_bothpredictions_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,prediction)
    push_pbiprediction_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,prediction)
    push_pbisample_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,Powerbi_sample)
    push_trainingdata_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,sample)
    push_metrics_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,parameters)
    gc.collect()

2018-04-20 00:36:44.265679
Producing Efficiency View for :  ADDataService
Opening memory : 
Total memory = 127.99956130981445; used memory = 44.82783889770508; available memory = 83.17172241210938
Memory after loading data : 
Total memory = 127.99956130981445; used memory = 44.983726501464844; available memory = 83.01583480834961
Memory after generating dates : 
Total memory = 127.99956130981445; used memory = 44.95704650878906; available memory = 83.04251480102539
% of Data between standard deviation range -1 & 1 : 0.7261885413754771
% of Data between standard deviation range -2 & 2 : 0.9388203757998912
% of Data between standard deviation range -3 & 3 : 0.9865695203794838
Number of rows after cleaning data: 1821085
Memory after filtering by standard deviation : 
Total memory = 127.99956130981445; used memory = 44.938262939453125; available memory = 83.06129837036133
Number of rows after filtering by influence: 1821085
sample count 1274759
Memory after filtering by influence and sampl

In [11]:
print(str(datetime.datetime.today()))
tenant='msods'
ibizaroles = ['adminwebservice']
for i in ibizaroles:
    print("Producing Efficiency View for : " , i)
    role = i
    metrics=[]
    sample,Powerbi_sample,prediction,parameters,path = efficiency(tenant,role,asofdate,storageaccountname,accountkey,container,No_PBI_samples,Sample_Percent,random_State,cv,cpu_bucket,memory_bucket,cpuvsrps_rpsbucket,cpuvsrps_rps_limit)
    push_bothpredictions_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,prediction)
    push_pbiprediction_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,prediction)
    push_pbisample_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,Powerbi_sample)
    push_trainingdata_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,sample)
    push_metrics_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,parameters)
    gc.collect()


2018-04-20 00:38:52.314059
Producing Efficiency View for :  ADExtension
Opening memory : 
Total memory = 127.99956130981445; used memory = 44.71616744995117; available memory = 83.28339385986328
Memory after loading data : 
Total memory = 127.99956130981445; used memory = 44.81513977050781; available memory = 83.18442153930664
Memory after generating dates : 
Total memory = 127.99956130981445; used memory = 44.80695343017578; available memory = 83.19260787963867
% of Data between standard deviation range -1 & 1 : 0.6470727486809946
% of Data between standard deviation range -2 & 2 : 0.9538142730829262
% of Data between standard deviation range -3 & 3 : 0.9981052960728838
Number of rows after cleaning data: 1129958
Memory after filtering by standard deviation : 
Total memory = 127.99956130981445; used memory = 44.7913818359375; available memory = 83.20817947387695
Number of rows after filtering by influence: 1129957
sample count 790969
Memory after filtering by influence and sampling : 

In [13]:
print(str(datetime.datetime.today()))
tenant='iamux'
roles = ['AzurePortalWebsite']
for i in roles:
    print("Producing Efficiency View for : " , i)
    role = i
    metrics=[]
    sample,Powerbi_sample,prediction,parameters,path = efficiency(tenant,role,asofdate,storageaccountname,accountkey,container,No_PBI_samples,Sample_Percent,random_State,cv,cpu_bucket,memory_bucket,cpuvsrps_rpsbucket,cpuvsrps_rps_limit)
    push_bothpredictions_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,prediction)
    push_pbiprediction_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,prediction)
    push_pbisample_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,Powerbi_sample)
    push_trainingdata_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,path,sample)
    push_metrics_toblob(storageaccountname,accountkey,tenant,role,container,asofdate,parameters)
    gc.collect()

2018-04-20 20:56:56.284492
Producing Efficiency View for :  AzurePortalWebsite
Opening memory : 
Total memory = 127.99956130981445; used memory = 46.25084686279297; available memory = 81.74871444702148
Memory after loading data : 
Total memory = 127.99956130981445; used memory = 46.51201629638672; available memory = 81.48754501342773
Memory after generating dates : 
Total memory = 127.99956130981445; used memory = 46.459190368652344; available memory = 81.54037094116211
% of Data between standard deviation range -1 & 1 : 0.664258342212415
% of Data between standard deviation range -2 & 2 : 0.9607406072389184
% of Data between standard deviation range -3 & 3 : 0.9977177413857344
Number of rows after cleaning data: 3205275
Memory after filtering by standard deviation : 
Total memory = 127.99956130981445; used memory = 46.42683029174805; available memory = 81.5727310180664
Number of rows after filtering by influence: 3205274
sample count 2243691
Memory after filtering by influence and sam